In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark.sql import SparkSession

# Crear o recuperar una SparkSession
spark = SparkSession.builder.master("local[*]").appName("WorldHappinessReport").getOrCreate()

# Leer el archivo CSV usando SparkSession
df = spark.read.option("header","true").csv('world-happiness-report-2021.csv', header=True, inferSchema="true")


In [3]:
df.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- Regional indicator: string (nullable = true)
 |-- Ladder score: double (nullable = true)
 |-- Standard error of ladder score: double (nullable = true)
 |-- upperwhisker: double (nullable = true)
 |-- lowerwhisker: double (nullable = true)
 |-- Logged GDP per capita: double (nullable = true)
 |-- Social support: double (nullable = true)
 |-- Healthy life expectancy: double (nullable = true)
 |-- Freedom to make life choices: double (nullable = true)
 |-- Generosity: double (nullable = true)
 |-- Perceptions of corruption: double (nullable = true)
 |-- Ladder score in Dystopia: double (nullable = true)
 |-- Explained by: Log GDP per capita: double (nullable = true)
 |-- Explained by: Social support: double (nullable = true)
 |-- Explained by: Healthy life expectancy: double (nullable = true)
 |-- Explained by: Freedom to make life choices: double (nullable = true)
 |-- Explained by: Generosity: double (nullable = true)
 |-- Explained 

In [4]:
df.createOrReplaceTempView("df")

## Apartado 1: país más feliz del 2021

In [5]:
mas_feliz=spark.sql("SELECT `Country name`, max(`Ladder score`) FROM df group by `Country name` order by max(`Ladder score`) desc")

In [6]:
mas_feliz.show()

+--------------+-----------------+
|  Country name|max(Ladder score)|
+--------------+-----------------+
|       Finland|            7.842|
|       Denmark|             7.62|
|   Switzerland|            7.571|
|       Iceland|            7.554|
|   Netherlands|            7.464|
|        Norway|            7.392|
|        Sweden|            7.363|
|    Luxembourg|            7.324|
|   New Zealand|            7.277|
|       Austria|            7.268|
|     Australia|            7.183|
|        Israel|            7.157|
|       Germany|            7.155|
|        Canada|            7.103|
|       Ireland|            7.085|
|    Costa Rica|            7.069|
|United Kingdom|            7.064|
|Czech Republic|            6.965|
| United States|            6.951|
|       Belgium|            6.834|
+--------------+-----------------+
only showing top 20 rows



El país más feliz del mundo en 2021 es Finlandia, seguido por u grupo de países europeos con alguna excepción. Es interesante ver cómo, por lo general, también en este parámetro se respeta el clásico norte - rico, sur - pobre. En términos de felicidad y bajo los parámetros medidos, el norte también es más feliz que el sur. También podríamos incidir en que países más fríos, pese a lo que se ha ido vendiendo amparado por términos como índice de suicidios, son más felices, aunque correlación no implica causalidad y puede que el clima sea una variable irrelevante frente a otras como PIB, renta mínima, sistema de salud, educación...

## Apartado 2: país más feliz por continente

In [7]:
mas_feliz_continente=spark.sql("SELECT `Regional indicator`, `Country name`, max(`Ladder score`) FROM df group by `Country name`,`Regional indicator` order by max(`Ladder score`) desc")

In [8]:
mas_feliz_continente.show(50,truncate=False)

+----------------------------------+------------------------+-----------------+
|Regional indicator                |Country name            |max(Ladder score)|
+----------------------------------+------------------------+-----------------+
|Western Europe                    |Finland                 |7.842            |
|Western Europe                    |Denmark                 |7.62             |
|Western Europe                    |Switzerland             |7.571            |
|Western Europe                    |Iceland                 |7.554            |
|Western Europe                    |Netherlands             |7.464            |
|Western Europe                    |Norway                  |7.392            |
|Western Europe                    |Sweden                  |7.363            |
|Western Europe                    |Luxembourg              |7.324            |
|North America and ANZ             |New Zealand             |7.277            |
|Western Europe                    |Aust

Se pueden observar los diferentes índices de felicidad de múltiples países ceñidos a regiones. Es complejo ordenar por continentes pues los datos vienen agrupados bajo ciertos parámetros que no corresponden estrictamente a continentes sino a zonas geográficas, por lo que habría que elaborar un listado de países por continente y cotejar o bien hacerlo manualmente obtenida la clasificación.  
El primer país europeo: Finlandia  
El primer país americano: Canadá  
El primer país asiático: Taiwán  
El primer país africano: Bahréin  
El primer país de oceanía: Nueva Zelanda

## Apartado 3: país que más veces ocupó el primer lugar en todos los años

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Inicializamos la sesión de spark y cargamos el CSV
spark = SparkSession.builder.appName("mas_feliz_n_veces").getOrCreate()
df_total = spark.read.option('header','true').csv('world-happiness-report.csv', header=True, inferSchema='true')

# Especificamos la ventana y asignamos números a las filas en función de la misma
windowSpec = Window.partitionBy("year").orderBy(F.desc("Life Ladder"))
df_total = df_total.withColumn("top_country", F.row_number().over(windowSpec))

# Filtramos por aquellos países que tienen un 1 asginado (han estado primeros al menos una vez)
mas_feliz_n_veces = df_total.filter(F.col("top_country") == 1)

# Agrupamos todos los filtrados por país
countFirstYear = mas_feliz_n_veces.groupBy("Country name", "top_country").agg(F.count("*").alias("country_count")).orderBy(F.desc("country_count"))

# Mostramos el resultado
countFirstYear.show()


+------------+-----------+-------------+
|Country name|top_country|country_count|
+------------+-----------+-------------+
|     Denmark|          1|            7|
|     Finland|          1|            6|
|      Norway|          1|            1|
| Switzerland|          1|            1|
|      Canada|          1|            1|
+------------+-----------+-------------+



El país que más veces ha estado en primera posición es Dinamarca con 7 apariciones, seguido de Finlandia con 6. De nuevo los países nórdicos al frente... ¡con el frío que hace ahí!

## Apartado 4: ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?

In [10]:
df_total = spark.read.option('header','true').csv('world-happiness-report.csv', header=True, inferSchema='true')
df_total.createOrReplaceTempView('df_total')
ordenados_por_GDP = spark.sql("SELECT `Country name`, max(`Log GDP per capita`) FROM df_total where year = 2020 group by `Country name` order by max(`Log GDP per capita`) desc")
ordenados_por_GDP.show(5)

+--------------------+-----------------------+
|        Country name|max(Log GDP per capita)|
+--------------------+-----------------------+
|             Ireland|                 11.323|
|         Switzerland|                 11.081|
|United Arab Emirates|                 11.053|
|              Norway|                 11.042|
|       United States|                 11.001|
+--------------------+-----------------------+
only showing top 5 rows



El país con el mayor GDP es Irlanda. Buscamos su posición en la lista de máxima felicidad

In [11]:
df_total = spark.read.option('header','true').csv('world-happiness-report.csv', header=True, inferSchema='true')
df_total.createOrReplaceTempView('df_total')
mas_feliz=spark.sql("SELECT `Country name`, max(`Life Ladder`) FROM df_total group by `Country name` order by max(`Life Ladder`) desc")
mas_feliz.show()

+-------------+----------------+
| Country name|max(Life Ladder)|
+-------------+----------------+
|      Denmark|           8.019|
|      Finland|           7.889|
|  Switzerland|           7.776|
|       Norway|           7.678|
|       Canada|            7.65|
|  Netherlands|           7.631|
|   Costa Rica|           7.615|
|  New Zealand|           7.604|
|      Iceland|           7.591|
|      Ireland|           7.568|
|       Sweden|            7.56|
|United States|           7.513|
|      Austria|           7.499|
|    Venezuela|           7.478|
|    Australia|            7.45|
|       Mexico|           7.443|
|       Israel|           7.433|
|   Luxembourg|           7.404|
|       Panama|           7.321|
|      Germany|           7.312|
+-------------+----------------+
only showing top 20 rows



Irlanda ocupa la posición 10 en la lista de países más felices.

# Artículo en Medium

Aquí está el [breve artículo escrito en Medium](https://medium.com/@alfsan04/producto-interior-bruto-o-%C3%ADndices-de-felicidad-ccc7661b0e02) respecto a la práctica elaborada.